In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
import glob
path = "./../dist/char*.txt"
contents=[]
for filename in glob.glob(path):
    with open(filename, 'r') as f:
        for line in f:
            contents.append(line.strip('\n'))
print('Liczba plików: ', len(contents)/15)
i=0
while (i<len(contents)-1):
    contents[i]=float(contents[i])
    i+=1
    if ((i+1)%15==0):
        i=i+1

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def normalize(x):
    min_v = torch.min(x)
    range_v = torch.max(x) - min_v
    if range_v > 0:
        return (x - min_v) / range_v
    else:
        return torch.zeros(x.size())

In [ ]:
from random import randint as r

In [ ]:
# while True:
#     a=r(0, (len(contents)/15))
#     print(a)
#     print(torch.tensor(contents[(a*15):(a*15+14)]))

In [ ]:
import time

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(in_features=14, out_features=10)
        self.fc2 = nn.Linear(in_features=10, out_features=4)
        #self.out = nn.Linear(in_features=14, out_features = 4, bias=True)
        
    def forward(self, t):
        #1st layer
        t = self.fc1(t)
        #2nd layer
        t = self.fc2(t)

        #t = F.softmax(t, -1)
        
        return t

In [ ]:
net=Network()
params=list(net.parameters())
criterion = nn.MSELoss()
#net.zero_grad()
optimizer = optim.SGD(net.parameters(), lr = 0.0001)#lr=0.00000000001)
lossDuplicationCounter = 0
prevloss=5
loss = 4
i = 0
good = 0

In [ ]:
begin=time.time()
while (good < 10) and lossDuplicationCounter<=30:
    if float(prevloss)<0.1: 
        good+=1
    else:
        good = 0
        
    a=r(0, (len(contents)/15-1))
    chars = normalize(torch.tensor(contents[(a*15):(a*15+14)]))
    out = net(chars)
    start=a*15
    #print(contents[start+14])
    if (contents[start+14]=='Marcin'):
        target = torch.tensor([1., 0., 0., 0.])
    elif (contents[start+14]=='Bartek'):
        target = torch.tensor([0., 1., 0., 0.])
    elif (contents[start+14]=='Maciek'):
        target = torch.tensor([0., 0., 1., 0.,])
    elif (contents[start+14]=='Artur'):
        target = torch.tensor([0., 0., 0., 1.,])
    loss = criterion(out, target)
    #print('loss', float(loss))
    if prevloss == loss:
        lossDuplicationCounter +=1
        #print('loss duplicate #', lossDuplicationCounter)
    else:
        lossDuplicationCounter = 0
    prevloss = loss
    optimizer.zero_grad()
    net.zero_grad()
    #loss.backward(retain_graph=True)
    loss.backward(retain_graph=True)
    loss.backward(retain_graph=True)
    loss.backward()
    optimizer.step()   # Does the update
       # zero the gradient buffers
    if i%1000==0:
        print("\n\niteration: ", i, 
              "\nLoss: ", ((str(loss)).strip('tensor()')).strip(', grad_fn=<MseLossBackward>) '), 
              '\nOutput: ', ((str(out)).strip('tensor(')).strip(', grad_fn=<AddBackward0>)').strip(', grad_fn=<Softmax'),
              '\nTarget: ', str(target).strip('tensor(').strip(')'))
    i+=1
print(i, "Learning time: \n\t", time.time()-begin)
torch.cuda.empty_cache()

In [ ]:
print("\n\niteration: ", i, 
              "\nLoss: ", ((str(loss)).strip('tensor()')).strip(', grad_fn=<MseLossBackward>) '), 
              '\nOutput: ', ((str(out)).strip('tensor(')).strip(', grad_fn=<AddBackward0>)').strip(', grad_fn=<Softmax'),
              '\nTarget: ', str(target).strip('tensor(').strip(')'))

In [ ]:
# print
# i=0
# for filename in glob.glob(path):
#     #a=r(0, (len(contents)/15-1))
#     chars = torch.tensor(contents[(i*15):(i*15+14)])
#     #print(str(net(chars)).strip(", grad_fn=<AddBackward0>)").strip("tensor("), "\t : ",contents[i*15+14])
#     print(((str(net(chars))).strip('tensor(')).strip(', grad_fn=<AddBackward0>)').strip(', grad_fn=<Softmax'), " : ", contents[i*15+14])
#     i=i+1

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
torch.save(net, "net.pt")


In [ ]:
haba = torch.load("net.pt")
haba.eval()


In [ ]:
i=0
ilezg=0
for filename in glob.glob(path):
    tt=normalize(torch.tensor(contents[(a*15):(a*15+14)]))
    #tt=F.softmax(normalize(torch.tensor(contents[(a*15):(a*15+14)])), -1)
    who = contents[i*15+14]
    predykcja = haba(tt)
    print(str(predykcja).strip('tensor(').strip(', grad_fn=<AddBackward0>)').strip(', grad_fn=<SoftmaxBackward>)')) 
    predlist = predykcja.tolist().index(max(predykcja.tolist()))
    predykcjaOsoba=''
    if predlist == 0:
        predykcjaOsoba='Marcin'
    elif predlist == 1:
        predykcjaOsoba='Bartek'
    elif predlist == 2:
        predykcjaOsoba='Maciek'
    elif predlist == 3:
        predykcjaOsoba='Artur'
    else:
        print('ERROR!!!')
    print(i, who, ' : \n', str(tt).strip('tensor(').strip(')').strip(', grad_fn=<Softmax')), #'\n', str(predykcja).strip('tensor(').strip(', grad_fn=<AddBackward0>)').strip(', grad_fn=<SoftmaxBackward>)'))
    i+=1
    if who == predykcjaOsoba:
        ilezg+=1
print(ilezg/i*100)